In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import VarianceThreshold

from sqlalchemy import MetaData, select
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session, registry
from sqlalchemy.exc import SQLAlchemyError

# from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

from utils.create_object import create_object
from utils.update_object_attributes import update_object_attributes
from utils.get_feature_id_by_name import get_feature_id_by_name

from utils.handle_engine import get_engine

In [ ]:
engine = get_engine()
metadata = MetaData()
metadata.reflect(bind=engine)
Base = automap_base(metadata=metadata)
Base.prepare(autoload_with=engine)
mapper_registry = registry()

### Load Data

#### Access tables

In [ ]:
metadata.tables["label"]

In [ ]:

datapoint = metadata.tables['datapoint']
datapoint_feature_value = metadata.tables['datapoint_feature_value']
feature = metadata.tables['feature']
datapoint_mappings = metadata.tables['datapoint_mappings']
datapoint_class_label = metadata.tables["datapoint_class_label"]
datapoint_rul_label = metadata.tables["datapoint_rul_label"]
label = metadata.tables["label"]


#### Construct the SQL query for training and test set

##### For datapoints

In [ ]:
# For train set
query_train = select(
    datapoint.c.id.label('datapoint_id'),
    feature.c.name.label('feature_name'),
    datapoint_feature_value.c.value.label('feature_value')
).select_from(
    datapoint.join(datapoint_feature_value, datapoint.c.id == datapoint_feature_value.c.datapoint_id)
    .join(feature, feature.c.id == datapoint_feature_value.c.feature_id)
    .join(datapoint_mappings, datapoint.c.datapoint_mappings_id == datapoint_mappings.c.id)
).where(
    datapoint_mappings.c.grouping == 'train'
)

# For train set
query_test = select(
    datapoint.c.id.label('datapoint_id'),
    feature.c.name.label('feature_name'),
    datapoint_feature_value.c.value.label('feature_value')
).select_from(
    datapoint.join(datapoint_feature_value, datapoint.c.id == datapoint_feature_value.c.datapoint_id)
    .join(feature, feature.c.id == datapoint_feature_value.c.feature_id)
    .join(datapoint_mappings, datapoint.c.datapoint_mappings_id == datapoint_mappings.c.id)
).where(
    datapoint_mappings.c.grouping == 'test'
)


##### For labels

In [ ]:

# Construct SQL query for categorical labels
query_cat_labels = select(
    datapoint.c.id.label('datapoint_id'),
    label.c.name.label('label_name'),
    datapoint_class_label.c.value.label('label_value')
).select_from(
    datapoint.join(datapoint_class_label, datapoint.c.id == datapoint_class_label.c.datapoint_id)
    .join(label, label.c.id == datapoint_class_label.c.label_id)
)

# Construct SQL query for continuous labels
query_cont_labels = select(
    datapoint.c.id.label('datapoint_id'),
    label.c.name.label('label_name'),
    datapoint_rul_label.c.value.label('label_value')
).select_from(
    datapoint.join(datapoint_rul_label, datapoint.c.id == datapoint_rul_label.c.datapoint_id)
    .join(label, label.c.id == datapoint_rul_label.c.label_id)
)


#### Execute the queries and load into DataFrames

##### Consider datapoint features

In [ ]:
df_train = pd.read_sql_query(query_train, engine)
df_test = pd.read_sql_query(query_test, engine)

# Pivot tables to get features as columns, for both train and test
train = df_train.pivot_table(index='datapoint_id', columns='feature_name', values='feature_value').reset_index()
test = df_test.pivot_table(index='datapoint_id', columns='feature_name', values='feature_value').reset_index()

##### Consider labels

In [ ]:
# Execute the queries and load into DataFrames
df_cat_labels = pd.read_sql_query(query_cat_labels, engine)
df_cont_labels = pd.read_sql_query(query_cont_labels, engine)

# Pivot table for categorical labels
pivot_cat_labels = df_cat_labels.pivot_table(
    index='datapoint_id',
    columns='label_name',
    values='label_value',
    aggfunc='first'  # Use 'first' or 'max' for categorical data
).reset_index()

# Pivot table for continuous labels
pivot_cont_labels = df_cont_labels.pivot_table(
    index='datapoint_id',
    columns='label_name',
    values='label_value',
    aggfunc='mean'  # Use 'mean' or another suitable function for numerical data
).reset_index()


In [ ]:
# Merge categorical and continuous labels
pivot_labels = pd.merge(pivot_cat_labels, pivot_cont_labels, on='datapoint_id', how='outer')

# Merge labels with feature dataframes
train_with_labels = pd.merge(train, pivot_labels, on='datapoint_id', how='left')
test_with_labels = pd.merge(test, pivot_labels, on='datapoint_id', how='left')

In [ ]:
test_with_labels.head()

In [ ]:
label_names = ["Risk Level", "Class"]
train_x = train_with_labels.drop(columns=label_names)
test_x = test_with_labels.drop(columns=label_names)
train_y = train_with_labels[label_names]
test_y = test_with_labels[label_names]

### Setup train process and train process parameters

In [ ]:
context = dict()
context["base"] = Base
context["session"] = Session(bind=engine)
session = context["session"]

In [ ]:
# Get table references
train_process_table = Base.classes["train_process"]
train_process_init_parameter_table = Base.classes["train_process_init_parameter"]

try:
    # Load initial parameters - use latest parameters if none were explicitly given
    count_process = session.query(train_process_table.id).count()
    count_paras = session.query(train_process_init_parameter_table.id).count()

    # Fetch the latest init_parameters safely
    init_parameters = session.query(train_process_init_parameter_table).order_by(train_process_init_parameter_table.id.desc()).first()

    # Clone and add init_parameters if counts are equal
    if count_process == count_paras and init_parameters is not None:
        init_parameters_clone = train_process_init_parameter_table()
        for column in train_process_init_parameter_table.__table__.columns:
            if column.name != "id":
                setattr(init_parameters_clone, column.name, getattr(init_parameters, column.name))
        session.add(init_parameters_clone)

    # Create and add new train_process entry
    train_process = create_object(context, "train_process", with_commit=True)

    # Create and add new train_process_score entry
    # Assuming a one-to-one relationship with train_process
    # train_process_score = create_object(context, "train_process_score", train_process_id=train_process.id)

    # TODO: What is train_process_Score?
    # Committing the changes
    session.commit()

    # Update initial parameters if available
    if init_parameters:
        update_object_attributes(init_parameters, train_process_id=train_process.id)

    session.commit()

except SQLAlchemyError as e:
    session.rollback()  # Rollback in case of error
    print(f"An error occurred: {e}")
finally:
    session.close()  # Ensure session is closed after the operation

In [ ]:
#Filtern von aussageschwachen Features
def get_signals_with_low_variance(df, threshold=0.10):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    cont_data = df.select_dtypes(include=numerics)
    selector = VarianceThreshold(threshold=threshold)
    selector.fit(cont_data)
    inverted_list = ~np.array(selector.get_support())
    return cont_data.columns[inverted_list].tolist()

def do_variance_filter(train_X, test_X, threshold=0.1):
    low_variant_signals = set()
    low_variant_signals.update(get_signals_with_low_variance(train_X), threshold=threshold)
    train_X_ = train_X.drop(columns=list(low_variant_signals))
    test_X_ = test_X.drop(columns=list(low_variant_signals))
    return train_X_, test_X_, low_variant_signals

In [ ]:
def sample_snapshot(train_X_, train_y_):
    value_counts = train_y_["Risk Level"].value_counts()
    for i in train_y_["Risk Level"].unique():
        drop_amount = value_counts[i]-value_counts.min()
        class_df = train_y_[train_y_["Risk Level"] == i]
        drop_indexes = class_df.sample(n=drop_amount).index
        train_y_ = train_y_.drop(drop_indexes)
        train_X_ = train_X_.drop(drop_indexes)
    used_indexes = train_X_.index
    train_y_ = train_y_.reset_index(drop=True)
    train_X_ = train_X_.reset_index(drop=True)
    return train_X_,train_y_, used_indexes


In [ ]:

def get_hyperopt_args():
    args = {
        "max_depth": hp.choice('max_depth', range(1,100)),
        "min_samples_leaf": hp.choice("min_samples_leaf", range(1,15)),
        "random_state": hp.randint("random_state", 3000),
        "max_features": hp.choice('max_features', range(1,50)),
        "criterion": hp.choice('criterion', ["gini", "entropy"]),
        "variance_threshold_var_fac": hp.randint("variance_threshold", 100),
        #"normalize": hp.choice('normalize', [0, 1])
    }
    return args

def do_train_run(train_X_:pd.DataFrame, train_y_:pd.DataFrame, test_X_:pd.DataFrame, test_y_:pd.DataFrame, args):    
    #Feature selection TODO: mutual info & correlation filter; performance
    #Variance Filter
    session = Session(engine)    
    threshold = args["variance_threshold_var_fac"]*args["variance_threshold_fac"]+args["variance_threshold_floor"]
    #args["hyperparameter"].threshold_feature_variance = threshold
    #args["hyperparameter"] = update_values(args["hyperparameter"], Base.classes["hyperparameter"], session, threshold_feature_variance = threshold)
    
    train_X_, test_X_, low_variant_signals = do_variance_filter(train_X_, test_X_, threshold)
    
    for i, signal in enumerate(low_variant_signals):
       dropped_feature_variance_filter = create_object(context, "dropped_feature_variance_filter", with_commit=True,
                                                       train_process_iteration_compute_result_id = args["train_process_iteration_compute_result"].id,
                                                       feature_id = get_feature_id_by_name(Base, signal))
       session.add(dropped_feature_variance_filter)
       session.commit()
    #args["hyperparameter"].max_depth = int(args["max_depth"])
    #args["hyperparameter"].min_samples_leaf = int(args["min_samples_leaf"])
    #args["hyperparameter"].random_state = int(args["random_state"])
    #args["hyperparameter"].max_features = int(args["max_features"])
    #args["hyperparameter"].criterion = args["criterion"]    
    update_values(args["hyperparameter"], Base.classes["hyperparameter"], session,
                    max_depth = int(args["max_depth"]),
                    min_samples_leaf = int(args["min_samples_leaf"]),
                    random_state = int(args["random_state"]),
                    max_features = int(args["max_features"]),
                    criterion = args["criterion"],                                                            
                    threshold_feature_variance = threshold
    )
    #session.commit()
    
    dtr = DecisionTreeClassifier(
        max_depth = args["max_depth"],
        min_samples_leaf = args["min_samples_leaf"],
        random_state = args["random_state"],
        max_features = args["max_features"],
        criterion = args["criterion"],
        #normalize = args["normalize"],
    )
    dtr.fit(train_X_, train_y_)
    eval_predict = dtr.predict(test_X_)    
    #TODO:Evaluate using full train data vs validation data or split test data...; intention:validation integrity, validation data is incorporated in trainings process
    accuracy = balanced_accuracy_score(test_y_, eval_predict) 
    #args["train_process_iteration_score"].balanced_accuracy_score = accuracy
    update_values(args["train_process_iteration_score"], Base.classes["train_process_iteration_score"], session, balanced_accuracy_score = accuracy)
    #session.commit()
    return accuracy

def train_run(args):
    train_X_ = args["train_X_"]
    train_y_ = args["train_y_"]
    test_X_ = args["test_X_"]
    test_y_ = args["test_y_"]
    return 1 - do_train_run(train_X_, train_y_, test_X_, test_y_, args)

def f(args):    
    acc = train_run(args)
    return {'loss': acc, 'status': STATUS_OK}

def do_train_process_iteration(args):
    trials = Trials()    
    #Create SQL Obj
    session = args["session"]
    hyperparameter = create_object(Base, "hyperparameter",
                                   id = getNextIdForTable(Base, "hyperparameter"))
    
    train_process_iteration = create_object(Base, "train_process_iteration",
                                            id = getNextIdForTable(Base, "train_process_iteration"),
                                            train_process_id = args["train_process"].id,
                                            hyperparameter_id = hyperparameter.id)
    
    train_process_iteration_score = create_object(Base, "train_process_iteration_score",
                                                id = getNextIdForTable(Base, "train_process_iteration_score"),
                                                train_process_iteration_id = train_process_iteration.id)
    
    train_process_iteration_compute_result = create_object(Base, "train_process_iteration_compute_result",
                                                           id = getNextIdForTable(Base, "train_process_iteration_compute_result"),
                                                           train_process_iteration_id = train_process_iteration.id)    
    session.add(hyperparameter)
    session.add(train_process_iteration)
    session.add(train_process_iteration_score)
    session.add(train_process_iteration_compute_result)
    session.commit()
    args["train_process_iteration"] = train_process_iteration
    args["hyperparameter"] = hyperparameter
    args["train_process_iteration_score"] = train_process_iteration_score
    args["train_process_iteration_compute_result"] = train_process_iteration_compute_result
    
    params = fmin(f, args, algo=tpe.suggest, max_evals = 10, trials=trials)
    print(params)
    

def do_train_prcess(train_process, paras):
    #Even out labels in train_data TODO: andere Methoden
    train_X_,train_y_, used_indexes = sample_snapshot(train_X, train_y)
    for i, index in enumerate(used_indexes):
        train_process_train_data_junction = create_object(Base, "train_process_train_data_junction",
                                                          train_process_id =  train_process.id,
                                                          train_data_id = index + 1)    
        session = Session(engine)
        session.add(train_process_train_data_junction)
        session.commit()   
        print(i/len(used_indexes))
    #Construct parameters
    variance_threshold_floor = paras.min_threshold_feature_variance
    variance_threshold_fac = paras.max_threshold_feature_variance - paras.min_threshold_feature_variance/100
    #Assign parameters  
    args = get_hyperopt_args()
    args["variance_threshold_floor"] = variance_threshold_floor
    args["variance_threshold_fac"] = variance_threshold_fac
    args["train_X_"] = train_X_
    args["train_y_"] = train_y_
    args["test_X_"] = test_x.reset_index(drop=True)
    args["test_y_"] = test_y.reset_index(drop=True)
    args["train_process"] = train_process
    args["session"] = Session(bind = engine)
    for i in range (4):
        args["index"] = i
        do_train_process_iteration(args)

do_train_prcess(train_process, init_parameters)

In [ ]:
#TODO: multithreaded implementation
def find_missing_elements(full_list, partial_list):
    missing_elements = []
    for element in full_list:
        if element not in partial_list:
            missing_elements.append(element)
    return missing_elements

def get_redundant_pairs(X_train: pd.DataFrame) -> set:
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = X_train.columns
    for i in range(0, X_train.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(X_train: pd.DataFrame, thresholds=[0.5]) -> list:
    au_corr = X_train.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(X_train)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    au_corrs = []
    for threshold in thresholds:
        au_corrs.append(au_corr[au_corr > threshold])
    return au_corrs

def get_indexes_to_drop(corr_df: pd.DataFrame, X_train: pd.DataFrame, y_train: pd.DataFrame) -> set:

    indexes_to_drop = set()
    le = LabelEncoder()
    y_train = pd.DataFrame(le.fit_transform(y_train), columns=[target_name])[target_name]

    for row in corr_df.index:
        if (X_train[row[0]].corr(y_train)) > (X_train[row[1]].corr(y_train)):
            indexes_to_drop.add(row[1])
        else:
            indexes_to_drop.add(row[0])
    
    return indexes_to_drop

def corr_filter(X_train: pd.DataFrame, y_train: pd.DataFrame, thresholds=[0.5]) -> list[pd.DataFrame]:
    remaining_dfs = []
    corr_dfs = get_top_abs_correlations(X_train, thresholds)
    for corr_df in corr_dfs:
        indexes_to_drop = get_indexes_to_drop(corr_df, X_train, y_train)
        remaining_df = X_train.drop(labels=indexes_to_drop, axis=1)
        remaining_dfs.append(remaining_df)
    return remaining_dfs

def find_best_threshold_corr(train_X, train_y, test_X, train_Y, target_name, thresholds):
    dfs = corr_filter(train_X, train_y, thresholds)
    best_threshold = None
    highest_accuracy = 0.0
    best_dropped = None
    for i, df in enumerate(dfs):
        filtered_X = df
        clf = DecisionTreeClassifier(random_state=0)
        clf.fit(filtered_X, train_y)        
        dropped = find_missing_elements(test_X.columns, filtered_X.columns)
        filtered_test = test_X.drop(columns=dropped)        
        y_pred = clf.predict(filtered_test)
        accuracy = balanced_accuracy_score(train_Y, y_pred)
        if accuracy > highest_accuracy:
            best_threshold = thresholds[i]
            highest_accuracy = accuracy
            best_dropped = dropped
    return best_dropped


def do_correlation_filter(train_X_, train_y_, test_X_, test_y_,):
    target_name = "Risk Level"
    cols_to_drop = find_best_threshold_corr(train_X, train_y[target_name], test_X, test_y[target_name], target_name, [0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95])